In [1]:
!pip install git+https://github.com/tensorflow/examples.git#egg=tensorflow-examples[model_maker]


  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-install-expkwg0f/tensorflow-examples
  Running command git clone -q https://github.com/tensorflow/examples.git /tmp/pip-install-expkwg0f/tensorflow-examples
     |████████████████████████████████| 323.7MB 50kB/s 
     |████████████████████████████████| 880kB 43.3MB/s 
     |████████████████████████████████| 1.1MB 57.8MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 266kB 44.3MB/s 
     |████████████████████████████████| 460kB 40.4MB/s 
     |████████████████████████████████| 6.7MB 40.4MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 174kB 57.1MB/s 
     |████████████████████████████████| 36.4MB 83kB/s 
     |████████████████████████████████| 358kB 52.1MB/s 
  Using cached https://files.pythonhosted.org/packages/89/e3/d576f6f02bc75bacbc3d42494e8f1d063c95617d86648dba243c2cb3963e/pybind11-2.5.0-py2.py3-none-any.whl
     |

In [2]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tensorflow_examples.lite.model_maker.core.data_util.text_dataloader import TextClassifierDataLoader
from tensorflow_examples.lite.model_maker.core.task.model_spec import AverageWordVecModelSpec
from tensorflow_examples.lite.model_maker.core.task import text_classifier

In [11]:
data_path = "/content/TextData"

In [12]:
model_spec = AverageWordVecModelSpec()

In [13]:
train_data = TextClassifierDataLoader.from_folder(os.path.join(data_path, 'train'), model_spec=model_spec, class_labels=['pos', 'neg'])
test_data = TextClassifierDataLoader.from_folder(os.path.join(data_path, 'test'), model_spec=model_spec, is_training=False, shuffle=False)

INFO:tensorflow:Saved vocabulary in /tmp/tmpe132ufqb/fda659d74838ec6d8f0b38ebef80e7f2_vocab.


INFO:tensorflow:Saved vocabulary in /tmp/tmpe132ufqb/fda659d74838ec6d8f0b38ebef80e7f2_vocab.


In [14]:
model = text_classifier.create(train_data, model_spec=model_spec, epochs=5)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/5
781/781 [==============================] - 4s 5ms/step - loss: 0.6470 - accuracy: 0.6635
Epoch 2/5
781/781 [==============================] - 4s 5ms/step - loss: 0.3557 - accuracy: 0.8584
Epoch 3/5
781/781 [==============================] - 3s 4ms/step - loss: 0.2814 - accuracy: 0.8906
Epoch 4/5
781/781 [==============================] - 3s 4ms/step - loss: 0.2474 - accuracy: 0.9033
Epoch 5/5
781/781 [==============================] - 3s 4ms/step - loss: 0.2274 - accuracy: 0.9135


In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 16)           160048    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 34        
Total params: 160,354
Trainable params: 160,354
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = model.evaluate(test_data)

782/782 [==============================] - 2s 3ms/step - loss: 0.3136 - accuracy: 0.8711


In [17]:
model.export(export_dir='.')

INFO:tensorflow:Saving labels in ./labels.txt.


INFO:tensorflow:Saving labels in ./labels.txt.


INFO:tensorflow:Assets written to: /tmp/tmpqbbyaoll/assets


INFO:tensorflow:Assets written to: /tmp/tmpqbbyaoll/assets


INFO:tensorflow:Saved vocabulary in ./vocab.


INFO:tensorflow:Saved vocabulary in ./vocab.


In [18]:
# Read TensorFlow Lite model from TensorFlow Lite file.
with tf.io.gfile.GFile('model.tflite', 'rb') as f:
  model_content = f.read()

# Read label names from label file.
with tf.io.gfile.GFile('labels.txt', 'r') as f:
  label_names = f.read().split('\n')

# Initialze TensorFlow Lite inpterpreter.
interpreter = tf.lite.Interpreter(model_content=model_content)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

# Run predictions on each test data and calculate accuracy.
accurate_count = 0
for text, label in test_data.dataset:
    # Add batch dimension and convert to float32 to match with the model's input
    # data format.
    text = tf.expand_dims(text, 0)

    # Run inference.
    interpreter.set_tensor(input_index, text)
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the label with highest
    # probability.
    predict_label = np.argmax(output()[0])
    # Get label name with label index.
    predict_label_name = label_names[predict_label]
    accurate_count += (predict_label == label.numpy())

accuracy = accurate_count * 1.0 / test_data.size
print('TensorFlow Lite model accuracy = %.4f' % accuracy)

TensorFlow Lite model accuracy = 0.8711
